In [2]:
!pip install xgboost

import numpy as np
import pandas as pd

import xgboost as xgb
from sklearn.metrics import accuracy_score

In [3]:
import os
DataPath = "E:\ISIC"
print(os.listdir(DataPath))

['ISIC_2020_Training_GroundTruth.csv', 'jpeg.zip', 'test', 'test.csv', 'tfrecords', 'tfrecords.zip', 'train', 'train.csv', 'train.csv.zip']


In [4]:
train= pd.read_csv(DataPath + '/train.csv')
test= pd.read_csv(DataPath + '/test.csv')
sub   = pd.read_csv(DataPath + '/sample_submission.csv')

print(train.head())

     image_name  patient_id     sex  age_approx anatom_site_general_challenge  \
0  ISIC_2637011  IP_7279968    male        45.0                     head/neck   
1  ISIC_0015719  IP_3075186  female        45.0               upper extremity   
2  ISIC_0052212  IP_2842074  female        50.0               lower extremity   
3  ISIC_0068279  IP_6890425  female        45.0                     head/neck   
4  ISIC_0074268  IP_8723313  female        55.0               upper extremity   

  diagnosis benign_malignant  target  
0   unknown           benign       0  
1   unknown           benign       0  
2     nevus           benign       0  
3   unknown           benign       0  
4   unknown           benign       0  


In [5]:
train['sex'] = train['sex'].fillna('na')
train['age_approx'] = train['age_approx'].fillna(0)
train['anatom_site_general_challenge'] = train['anatom_site_general_challenge'].fillna('na')

test['sex'] = test['sex'].fillna('na')
test['age_approx'] = test['age_approx'].fillna(0)
test['anatom_site_general_challenge'] = test['anatom_site_general_challenge'].fillna('na')

In [6]:
train['sex'] = train['sex'].astype("category").cat.codes +1
train['anatom_site_general_challenge'] = train['anatom_site_general_challenge'].astype("category").cat.codes +1
train.head()

,image_name,patient_id,sex,age_approx,anatom_site_general_challenge,diagnosis,benign_malignant,target
0,ISIC_2637011,IP_7279968,2,45.0,1,unknown,benign,0
1,ISIC_0015719,IP_3075186,1,45.0,7,unknown,benign,0
2,ISIC_0052212,IP_2842074,1,50.0,2,nevus,benign,0
3,ISIC_0068279,IP_6890425,1,45.0,1,unknown,benign,0
4,ISIC_0074268,IP_8723313,1,55.0,7,unknown,benign,0


In [7]:
test['sex'] = test['sex'].astype("category").cat.codes +1
test['anatom_site_general_challenge'] = test['anatom_site_general_challenge'].astype("category").cat.codes +1
test.head()

,image_name,patient_id,sex,age_approx,anatom_site_general_challenge
0,ISIC_0052060,IP_3579794,2,70.0,3
1,ISIC_0052349,IP_7782715,2,40.0,2
2,ISIC_0058510,IP_7960270,1,55.0,6
3,ISIC_0073313,IP_6375035,1,50.0,6
4,ISIC_0073502,IP_0589375,1,45.0,2


In [8]:
x_train = train[['sex', 'age_approx','anatom_site_general_challenge']]
y_train = train['target']


x_test = test[['sex', 'age_approx','anatom_site_general_challenge']]
# y_train = test['target']


train_DMatrix = xgb.DMatrix(x_train, label= y_train)
test_DMatrix = xgb.DMatrix(x_test)

In [9]:
param = {
    'booster':'gbtree', 
    'eta': 0.3,
    'num_class': 2,
    'max_depth': 8
}

epochs = 100

In [10]:
clf = xgb.XGBClassifier(n_estimators=1000, 
                        max_depth=8, 
                        objective='multi:softprob',
                        seed=0,  
                        nthread=-1, 
                        learning_rate=0.015,
                        num_class = 2, 
                        scale_pos_weight = (32542/584))

In [11]:
clf.fit(x_train, y_train)

[23:45:23] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { scale_pos_weight } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.015, max_delta_step=0, max_depth=8,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=1000, n_jobs=-1, nthread=-1, num_class=2,
              num_parallel_tree=1, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=55.72260273972603,
              seed=0, subsample=1, tree_method='exact', validate_parameters=1,
              verbosity=None)

In [12]:
pred_train = clf.predict(x_train)

In [13]:
accuracy_score(pred_train, y_train)

0.9825212823763811

In [14]:
# proba = model.predict_proba(test_DMatrix) 
# clf.predict_proba(x_test)[:,1]
# clf.predict(x_test)
sub.target = clf.predict_proba(x_test)[:,1]
sub_tabular = sub.copy()